In [1]:
import boto3
import re
import urllib.request as req
import os
from terraaws.etl import JSON

In [11]:
from terraaws.session import BotoSession

In [1]:
bsm.profile_name

NameError: name 'bsm' is not defined

In [12]:
bsm = BotoSession(profile_name='mjn')

In [1]:
from terraaws.utilities import S3

In [2]:
s3 = S3(profile_name = 'mjn')

In [3]:
s3.create_private_bucket('taco-burrito')

{'ResponseMetadata': {'RequestId': '16RWRPVYA1SG8QQB',
  'HostId': '9XKyE3BVMjL4L5E9E751bGS/XuX2gqaBW40UMwFx2rlkR13zfF+KQq7ELjIhZKT5zowARLf+CtI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9XKyE3BVMjL4L5E9E751bGS/XuX2gqaBW40UMwFx2rlkR13zfF+KQq7ELjIhZKT5zowARLf+CtI=',
   'x-amz-request-id': '16RWRPVYA1SG8QQB',
   'date': 'Mon, 29 Mar 2021 16:00:43 GMT',
   'location': 'http://taco-burrito.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://taco-burrito.s3.amazonaws.com/'}

In [4]:
s3.session.region_name

'us_west-2'

In [8]:
s3 = s3.session.client('s3')

InvalidRegionError: Provided region_name 'us_west-2' doesn't match a supported format.

In [10]:
s3.session.profile_name

'default'

In [3]:
def get_public_ip():
    data = str(req.urlopen('http://checkip.dyndns.com/').read())
    
    return re.compile(r'Address: (\d+.\d+.\d+.\d+)').search(data).group(1)


def create_env(boto_session, cidr, name):
    ec2 = boto_session.client('ec2')
    #vpc
    vpc = ec2.create_vpc(CidrBlock = cidr, 
                         TagSpecifications=[{'ResourceType': 'vpc', 'Tags': [{'Key': 'Name', 'Value': name + '-vpc'}]}])
    vpc_id = vpc['Vpc']['VpcId']
    waiter = ec2.get_waiter('vpc_available')
    waiter.wait(VpcIds=[vpc_id])
    _ = ec2.modify_vpc_attribute(VpcId = vpc_id,
                                 EnableDnsHostnames = {'Value': True})
    

    #default security group, open internal traffic
    ip22 = get_public_ip() + '/32'
    sg = ec2.describe_security_groups(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    sg_id = sg['SecurityGroups'][0]['GroupId']
    sg_tag = name + '-base-securitygroup'
    int_sg_tag = ec2.create_tags(Resources=[sg_id], Tags=[{'Key': 'Name', 'Value': sg_tag}])       
    ingress = ec2.authorize_security_group_ingress(GroupId=sg_id,
                                                   IpPermissions=[
                                                       {'IpProtocol': 'tcp',
                                                        'FromPort': 80,
                                                        'ToPort': 80,
                                                        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                                       {'IpProtocol': 'tcp',
                                                        'FromPort': 443,
                                                        'ToPort': 443,
                                                        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                                       {'IpProtocol': 'tcp',
                                                        'FromPort': 22,
                                                        'ToPort': 22,
                                                        'IpRanges': [{'CidrIp': ip22}]}
                                                   ])

    
    #default route table
    rt = ec2.describe_route_tables(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    rt_id = rt['RouteTables'][0]['Associations'][0]['RouteTableId']
    rt_tag = name + '-routetable'
    rt_tbl_tag = ec2.create_tags(Resources=[rt_id], Tags=[{'Key': 'Name', 'Value': rt_tag}])
    
    #describe
    vpc = ec2.describe_vpcs(VpcIds=[vpc_id])
    sg = ec2.describe_security_groups(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    rt = ec2.describe_route_tables(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
    
    return vpc, sg, rt 


def create_subnet(boto_session, vpc, cidr, az, name, public=False):
    ec2 = boto_session.client('ec2')
    subnet = ec2.create_subnet(VpcId = vpc['Vpc']['VpcId'],
                            CidrBlock = cidr,
                            AvailabilityZone = az,
                            TagSpecifications=[{'ResourceType': 'subnet', 'Tags': [{'Key': 'Name','Value': name}]}])
    #auto assign ip
    if public == True:
        pub_sub = ec2.modify_subnet_attribute(SubnetId = subnet['Subnet']['SubnetId'],
                                          MapPublicIpOnLaunch = {'Value': True})
             
    #describe
    subnet = ec2.describe_subnets(Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
            
    return subnet


def create_internet_gateway(boto_session, vpc, name):
    ec2 = boto_session.client('ec2')
    ig = ec2.create_internet_gateway(TagSpecifications=[{'ResourceType': 'internet-gateway', 'Tags': [{'Key': 'Name','Value': name}]}])
    ig_attach = ec2.attach_internet_gateway(InternetGatewayId = ig['InternetGateway']['InternetGatewayId'],
                                     VpcId=vpc['Vpc']['VpcId'])

    #describe
    ig = ec2.describe_(Filters=[{'Name': 'internet-gateway-id',
                                 'Values': ig['InternetGateway']['InternetGatewayId']}])
    
    return ig

def create_internet_route(boto_session, route_table, cidr, gateway):
    ec2 = boto_session.client('ec2')
    rt = ec2.create_route(RouteTableId = route_table['RouteTables'][0]['RouteTableId'],
                          DestinationCidrBlock = cidr,
                          GatewayId = gateway['InternetGateway']['InternetGatewayId'])
    #describe  
    rt = ec2.describe_route_tables(RouteTableIds=[route_table['RouteTables'][0]['RouteTableId']])
            
    return rt


def create_db_subnet_group(boto_session, db_identifier, subnet_ids):
    rds = boto_session.client('rds')
    dbsubnet = rds.create_db_subnet_group(DBSubnetGroupDescription='{db} subnet group'.format(db=db_identifier),
                                          DBSubnetGroupName='{db}-subnetgroup'.format(db=db_identifier),
                                          SubnetIds=subnet_ids,
                                          Tags=[{'Key': 'Name','Value': '{db}-subnetgroup'.format(db=db_identifier)}])
    #describe
    dbsubnet = rds.describe_db_subnet_groups(Filters=[{'Name': 'DBSubnetGroupName',
                                                       'Values': ['{db}-subnetgroup'.format(db=db_identifier)]}])

    return dbsubnet


def create_key(boto_session, key_name, path):
    ec2 = boto_session.client('ec2')
    priv_key = ec2.create_key_pair(KeyName = key_name)
    key_file = open(path + os.sep + key_name + '.pem', "w")
    f = key_file.write(priv_key['KeyMaterial'])
    key_file.close()
    
    return priv_key
    

def launch_amazon_linux2(boto_session, name, subnet_id, security_groups, key_name, 
                         itype = 't2.micro', ami = 'ami-05b622b5fa0269787'):
    ec2 = boto_session.client('ec2')
    instance = ec2.run_instances(ImageId = ami,
                     InstanceType = itype,
                     MaxCount = 1,
                     MinCount = 1,
                     SubnetId = subnet_id,
                     BlockDeviceMappings = [{'DeviceName': '/dev/xvda',
                                           'VirtualName': name + '-storage',
                                           'Ebs': {'DeleteOnTermination': True,
                                                   'VolumeSize': 30}
                                          }],
                     TagSpecifications = [{'ResourceType': 'instance', 
                                         'Tags': [{'Key': 'Name','Value': name}]}],
                     SecurityGroupIds = security_groups,
                     KeyName = key_name)
    waiter = ec2.get_waiter('instance_status_ok')
    waiter.wait(InstanceIds=[instance['Instances'][0]['InstanceId']])

    #describe
    instance = ec2.describe_instances(InstanceIds=[instance['Instances'][0]['InstanceId']])
    
    return instance

def launch_mysql_rds(dbname, username, masterpwd, dbsubnetgrp, securitygroup):
    rds = bsm.client('rds')
    db = rds.create_db_instance(DBName = dbname,
                                         DBInstanceIdentifier = dbname,
                                         AllocatedStorage = 20,
                                         DBInstanceClass = 'db.t2.micro',
                                         Engine = 'mysql',
                                         MasterUsername = username,
                                         MasterUserPassword = masterpwd,
                                         PubliclyAccessible = True,
                                         StorageType = 'gp2',
                                         StorageEncrypted = False,
                                         AutoMinorVersionUpgrade = True,
                                         MultiAZ = False,
                                         DBSubnetGroupName = dbsubnetgrp,
                                         VpcSecurityGroupIds=[securitygroup],
                                         Tags=[{'Key': 'Name', 'Value': dbname}])
    waiter = rds.get_waiter('db_instance_available')
    waiter.wait(DBInstanceIdentifier = db['DBInstance']['DBInstanceIdentifier'])
    
    #describe
    db = rds.client.describe_db_instances(DBInstanceIdentifier=db['DBInstance']['DBInstanceIdentifier'])
                                                                  
    return db


In [10]:
bsm = boto3.Session(profile_name = 'mjn', region_name = 'us-west-2')

In [2]:
bsm = BotoSession(profile_name = 'mjn', region_name = 'us-west-2')

In [3]:
s3 = bsm.session.client

TypeError: 'Session' object is not callable

In [11]:
s3 = bsm.client('s3')
bucket = s3.create_bucket(ACL='private', Bucket='burrito-taco',
                          CreateBucketConfiguration = {'LocationConstraint': 'us-west-2'})

In [4]:
prj_vpc, prj_sg, prj_rt = create_env(bsm, '213.213.48.0/24', 'tripoli')

In [5]:
dbsubnet01 = create_subnet(bsm, prj_vpc, '213.213.48.32/27', 'us-west-2b', 'tripoli-dbsubnet-01')
dbsubnet02 = create_subnet(bsm, prj_vpc, '213.213.48.64/27', 'us-west-2c', 'tripoli-dbsubnet-02')
publicsubnet = create_subnet(bsm, prj_vpc, '213.213.48.96/27', 'us-west-2a', 'tripoli-publicsubnet', public = True)

In [6]:
intgateway = create_internet_gateway(bsm, prj_vpc, 'tripoli-ig')
int_route = create_internet_route(bsm, prj_rt, '0.0.0.0/0', intgateway)

In [8]:
prj_ec2 = launch_amazon_linux2(bsm, 'tripoli-ec2', publicsubnet['Subnet']['SubnetId'], 
                                   [prj_sg['SecurityGroups'][0]['GroupId']],
                                  'tripoli_key')

In [9]:
dbsubnetgrp = create_db_subnet_group(bsm, 'tripolidb', 
                                     [dbsubnet01['Subnet']['SubnetId'], dbsubnet02['Subnet']['SubnetId']])

In [12]:
prj_db = launch_mysql_rds('*****', '*****', '*****', 
                 dbsubnetgrp['DBSubnetGroup']['DBSubnetGroupName'], 
                 prj_sg['SecurityGroups'][0]['GroupId'])